# Routing Analysis

### Dependencies

In [53]:
# Data processing
import numpy as np
import pandas as pd
import geopandas as gpd
from datetime import datetime
import simpledbf as sdbf

# Visualisation
import matplotlib
from ipyleaflet import Map, GeoData, basemaps, LayersControl
import seaborn
import folium
import mapclassify

#API Handling
import simplejson
import urllib.request as ur
import routingpy as rp
import googlemaps
KEY = ""

### Loading Files

In [54]:
# Postal codes database
buildings = gpd.read_file('Generated Files/buildings_EPSG3414_no-dups.shp')

# Amenities
activesg = gpd.read_file('Generated Files/activesg.shp')
bus_stops = gpd.read_file('Generated Files/busstopssg_EPSG3414.shp')
hawkercentres = gpd.read_file('Generated Files/hawkercentres_EPSG3414.shp')
malls = gpd.read_file('Generated Files/malls_EPSG3414.shp')
parks = gpd.read_file('Generated Files/parks_EPSG3414.shp')
schools = gpd.read_file('Generated Files/schools_EPSG3414.shp')

In [6]:
len(buildings)

121361

In [7]:
# buildings.explore(legend=False)

### Trying ways to make API calls

In [8]:
orig = "3155CommonwealthAveWest"
dest = "16CollegeAvenueWest"
url = "https://maps.googleapis.com/maps/api/distancematrix/json?key={0}&origins={1}&destinations={2}&mode=driving&language=en-EN&sensor=false".format(KEY,str(orig),str(dest))
result= simplejson.load(ur.urlopen(url))
#print(result)
driving_time = result['rows'][0]['elements'][0]['duration']['text']
print(driving_time)

6 mins


In [17]:
#Amenity search
schools.head()
schools[schools['ADDRESS'].str.contains('CLEMENTI')]


,Unnamed_ 0,ADDRESS,BLK_NO,BUILDING,LATITUDE,LONGITUDE,LONGTITUDE,POSTAL,ROAD_NAME,SEARCHVAL,X,Y,address_1,geometry
24,0,15 CLEMENTI AVENUE 5 PEI TONG PRIMARY SCHOOL S...,15,PEI TONG PRIMARY SCHOOL,1.316148,103.767576,103.767576,129857,CLEMENTI AVENUE 5,PEI TONG PRIMARY SCHOOL,20683.51520,33158.57465,Pei Tong Primary School,POINT (20683.515 33158.575)
87,0,8 CLEMENTI AVENUE 3 CLEMENTI PRIMARY SCHOOL SI...,8,CLEMENTI PRIMARY SCHOOL,1.315809,103.763055,103.763055,129903,CLEMENTI AVENUE 3,CLEMENTI PRIMARY SCHOOL,20180.32316,33121.05322,Clementi Primary School,POINT (20180.323 33121.053)


### Trying routingpy, doesn't work

In [42]:
#YNC LAT LONG 1.306250 103.771890
#CLEMENTI PRIMARY SCHOOL 1.315809 103.763055

YNC = [103.771890, 1.306250]
ClemPriSchool = [103.763055, 1.315809]

router = rp.get_router_by_name("google")(api_key=KEY)
print(router)

search = router.matrix(locations = [[103.771890, 1.306250], [103.763055, 1.315809]], profile='drivng')

In [47]:
#Seems to be broken for transit so we can't use routingpy
searchTransit = router.matrix(locations = [[103.771890, 1.306250], [103.763055, 1.315809]], profile='transit', transit_mode=['bus','subway'], region='sg', units='metric' )

KeyError: 'duration'

In [44]:
search.durations[1]
print(search)

Matrix([[0, 484], [490, 0]], [[0, 2106], [2571, 0]])


### Using GoogleMaps Library

In [56]:
YNC = (1.306250, 103.771890)
ClemPriSchool = (1.315809, 103.763055)

gmaps = googlemaps.Client(key=KEY)

# Geocoding an address
geocode_result = gmaps.geocode('1600 Amphitheatre Parkway, Mountain View, CA')

# Look up an address with reverse geocoding
reverse_geocode_result = gmaps.reverse_geocode((40.714224, -73.961452))

# Request directions via public transit
now = datetime.now()
directions_result = gmaps.distance_matrix(origins = "16 College Avenue West, Singapore",
                                     destinations = "133 Clarence Lane, Singapore",
                                     mode="transit",
                                     departure_time=now)

In [126]:
schools["latlong"] = tuple(zip(schools.LATITUDE, schools.LONGITUDE))
allschools = list(schools.latlong)
allschoolsFirst20 = allschools[0:20]

In [85]:
now = datetime.now()
directions_result = gmaps.distance_matrix(origins = [(1.306250, 103.771890)],
                                     destinations = [(1.315809, 103.763055), (1.316148, 103.767576)],
                                     mode="transit",
                                     departure_time=now)
print(directions_result)

{'destination_addresses': ['8 Clementi Ave 3, Singapore 129903', '15 Clementi Ave 5, Singapore 129857'], 'origin_addresses': ['10 College Ave West, Singapore'], 'rows': [{'elements': [{'distance': {'text': '2.5 km', 'value': 2503}, 'duration': {'text': '17 mins', 'value': 1031}, 'status': 'OK'}, {'distance': {'text': '2.5 km', 'value': 2513}, 'duration': {'text': '18 mins', 'value': 1065}, 'status': 'OK'}]}], 'status': 'OK'}


In [128]:
#Measuring YNC accessibility to all schools
directions_result = gmaps.distance_matrix(origins = [(1.306250, 103.771890)],
                                     destinations = allschoolsFirst20,
                                     mode="transit",
                                     departure_time=now)
print(directions_result)

{'destination_addresses': ['Princess Elizabeth Primary School, 30 Bukit Batok West Avenue 3 (S)659163, Singapore 659163', '2 Woodlands Drive 14, Singapore 738079', '51 Coronation Rd, Singapore 269460', "52 King's Rd, Singapore 268097", '70 Punggol Dr., Waterway Primary School, Singapore 828802', '2A Prince Charles Cres, Singapore 159016', '80 Bartley Rd, Singapore 539786', '25 Mount Vernon Rd, Singapore 368051', '18 Bukit Batok Street 21, Singapore 659634', '11 Woodlands Ring Rd, Singapore 738240', '2 Bukit Batok Street 52, Singapore 659243', '2 Yishun Street 71, Singapore 768515', '71 Anchorvale Link, Singapore 544799', '10 Bishan Street 13, Singapore 579795', '8 Bishan Street 13, Singapore 579793', '10 Yishun Street 11, Singapore 768643', '4 Chestnut Dr, Singapore 679287', '2 Farrer Park Rd, Singapore 217567', '10 Seraya Rd, Singapore 437259', '1.32434093,103.9391298'], 'origin_addresses': ['10 College Ave West, Singapore'], 'rows': [{'elements': [{'distance': {'text': '8.2 km', 'val

In [146]:
#Getting accessibility score
len(directions_result["rows"][0]["elements"])

def accessScore(ans):
    sum = 0
    for i in range(len(ans["rows"][0]["elements"])-1):
        sum += ans["rows"][0]["elements"][i]['duration']['value']
    print(str(round(sum/len(ans["rows"][0]["elements"])/60, 2))+'min')

accessScore(directions_result)

53.44min


In [147]:
smu_result = gmaps.distance_matrix(origins = "Singapore Management University, Singapore",
                                     destinations = allschoolsFirst20,
                                     mode="transit",
                                     departure_time=now)

In [148]:
accessScore(smu_result)

43.52min


In [150]:
def AccessScorer(start, amenity):
    result = gmaps.distance_matrix(start,
                                     destinations = amenity,
                                     mode="transit",
                                     departure_time=now)
    sum = 0
    for i in range(len(result["rows"][0]["elements"])-1):
        sum += result["rows"][0]["elements"][i]['duration']['value']
    print(str(round(sum/len(result["rows"][0]["elements"])/60, 2))+'min')

AccessScorer("Singapore Mangament University, Singapore", allschoolsFirst20)

43.55min


In [109]:
directions_result["rows"][0]["elements"][1]
pdjson = pd.json_normalize(directions_result)
pdjson

,destination_addresses,origin_addresses,rows,status
0,"[8 Clementi Ave 3, Singapore 129903, 15 Clemen...","[10 College Ave West, Singapore]","[{'elements': [{'distance': {'text': '2.5 km',...",OK


In [63]:
test = gmaps.geocode("16 College Avenue West, Singapore")
print(test)

[{'address_components': [{'long_name': '16', 'short_name': '16', 'types': ['street_number']}, {'long_name': 'College Avenue West', 'short_name': 'College Ave West', 'types': ['route']}, {'long_name': 'Queenstown', 'short_name': 'Queenstown', 'types': ['neighborhood', 'political']}, {'long_name': 'Singapore', 'short_name': 'Singapore', 'types': ['locality', 'political']}, {'long_name': 'Singapore', 'short_name': 'SG', 'types': ['country', 'political']}], 'formatted_address': '16 College Ave West, Singapore', 'geometry': {'location': {'lat': 1.3077589, 'lng': 103.7721908}, 'location_type': 'ROOFTOP', 'viewport': {'northeast': {'lat': 1.308607630291502, 'lng': 103.7733037302915}, 'southwest': {'lat': 1.305909669708498, 'lng': 103.7706057697085}}}, 'place_id': 'ChIJ7-dK-PQa2jERVTxXIZWqbxU', 'plus_code': {'compound_code': '8Q5C+4V Singapore', 'global_code': '6PH58Q5C+4V'}, 'types': ['street_address']}]


In [68]:
test = pd.DataFrame(test)

In [78]:
test.columns
test.geometry

0    {'location': {'lat': 1.3077589, 'lng': 103.772...
Name: geometry, dtype: object

In [21]:
poly = gpd.read_file(r'Data Sources\1_data\subzone-census-2010\Subzone_Census2010.shp')
CCK_geo = poly.loc[poly['PLN_AREA_N'] == 'CHOA CHU KANG', 'geometry']
CCK_geo

282    POLYGON ((19097.664 41203.632, 19100.357 41203...
283    POLYGON ((18569.141 40067.008, 18997.336 39876...
284    POLYGON ((19044.893 42741.955, 19044.597 42741...
285    POLYGON ((19097.664 41203.632, 19079.502 41208...
286    POLYGON ((19079.502 41208.068, 19083.066 41183...
287    POLYGON ((18443.314 42028.961, 18427.736 41985...
288    POLYGON ((17985.936 40383.451, 17967.919 40371...
Name: geometry, dtype: geometry

In [7]:
df = subzones = gpd.read_file('Generated Files/subzones_featurecount.shp')

dfcck = df.loc[df['PLN_AREA_N'] == 'CHOA CHU KANG', 'geometry']
dfcck



282    POLYGON ((19097.664 41203.632, 19100.357 41203...
283    POLYGON ((18569.141 40067.008, 18997.336 39876...
284    POLYGON ((19044.893 42741.955, 19044.597 42741...
285    POLYGON ((19097.664 41203.632, 19079.502 41208...
286    POLYGON ((19079.502 41208.068, 19083.066 41183...
287    POLYGON ((18443.314 42028.961, 18427.736 41985...
288    POLYGON ((17985.936 40383.451, 17967.919 40371...
Name: geometry, dtype: geometry

In [22]:
print(len(activesg))
print(len(bus_stops))
print(len(hawkercentres))
print(len(malls))
print(len(parks))
print(len(schools))
print(len(df))


35
5072
125
159
352
181
311


In [12]:
parks.head()

,LANDXADDRE,LANDYADDRE,NAME,DESCRIPTIO,INC_CRC,FMEL_UPD_D,geometry
0,20449.4258,36630.1900,Bukit Batok Nature Park,Along Bukit Batok East Avenue 2 and Bukit Bato...,687BC3283CECEB71,2021/01/22,POINT (20449.426 36630.190)
1,21882.5449,38885.2200,Zhenghua Nature Park,"Bounded by Bt Timah Expressway (BKE), BKE Slip...",E879D84E9F03FC10,2021/01/22,POINT (21882.545 38885.220)
2,21388.3000,38307.4766,Dairy Farm Nature Park,Along Dairy Farm Road,D2E81816E37D78B7,2021/01/22,POINT (21388.300 38307.477)
3,16630.9316,35360.1055,Jurong Lake Gardens,Along Yuan Ching Road and Boon Lay Way,DB390E82DD9B7A6F,2021/01/22,POINT (16630.932 35360.105)
4,29376.9434,30516.8457,The Foothills Fort Canning Park,"Along River Valley Road, opposite Liang Court",99BE2CEDB1A1BFA8,2020/02/18,POINT (29376.943 30516.846)


In [47]:
df["centroid"] = df.geometry.centroid
df["centroid"].explore()

C:\Users\seanm\AppData\Roaming\Python\Python310\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [ ]:
df2 = df.to_crs({'init': 'epsg:4326'})
df2['centroid'] = df["centroid"].to_crs({'init': 'epsg:4326'})
df2

In [ ]:
# Amenities
activesg = activesg.to_crs({'init': 'epsg:4326'})
bus_stops = bus_stops.to_crs({'init': 'epsg:4326'})
hawkercentres = hawkercentres.to_crs({'init': 'epsg:4326'})
malls = malls.to_crs({'init': 'epsg:4326'})
parks = parks.to_crs({'init': 'epsg:4326'})
schools = schools.to_crs({'init': 'epsg:4326'})

In [62]:
origins = [(1.306250, 103.771890)]
mallsIter = (len(malls)%24, ((len(malls)-len(malls)%24)/24))
mallsIter
schools["latlong"] = tuple(zip(schools.LATITUDE, schools.LONGITUDE))
allschools = list(schools.latlong)
allschoolsFirst20 = allschools[0:20]

(15, 6.0)

In [72]:
from math import remainder


def AccessScorer(start, amenity):
    result = gmaps.distance_matrix(start,
                                     destinations = amenity,
                                     mode="transit",
                                     departure_time=now)
    sum = 0
    for i in range(len(result["rows"][0]["elements"])-1):
        sum += result["rows"][0]["elements"][i]['duration']['value']
    print(str(round(sum/len(result["rows"][0]["elements"])/60, 2))+'min')

# AccessScorer("Singapore Mangament University, Singapore", allschoolsFirst20)

def allscore(index, subzones, skip, ammenities):
    malls, hawkercentres, schools = ammenities[0], ammenities[1], ammenities[2]
    malls = malls.loc[[i for i in range(0,len(malls),skip)]]
    malls["latlong"] = tuple(zip(malls.geometry.x, malls.geometry.y))
    hawkercentres = hawkercentres.loc[[i for i in range(0,len(malls),skip)]]
    hawkercentres["latlong"] = tuple(zip(hawkercentres.geometry.x, hawkercentres.geometry.y))
    schools = schools.loc[[i for i in range(0,len(malls),skip)]]
    schools["latlong"] = tuple(zip(schools.geometry.x, schools.geometry.y))
    mallsIter = (len(malls)%24, ((len(malls)-len(malls)%24)/24))
    hawkercentresIter = (len(hawkercentres)%24, ((len(hawkercentres)-len(hawkercentres)%24)/24))
    schoolsIter = (len(schools)%24, ((len(schools)-len(schools)%24)/24))
    subzones = subzones.loc[index]
    def calc(subzones, ammenity, ammenityIter):
        subzones['access'] = 0
        total = 0
        for i in range(0, ammenityIter[0]+2):
            if i !=  ammenityIter[0] + 1:
                curAmmenity = ammenity.loc[i in range(i*24,i*24 + 24)]
                for i in range(0,len(subzones)):
                    subzones.loc['access',i] = AccessScorer([i.geometry.y, i.geomtry.x], )
            else:
                curAmmenity = ammenity.loc[i in range(i*24,i*24 + ammenityIter[0])]
                for i in range(0,len(subzones)):
                    subzones.loc['access',i] = AccessScorer([i.geometry.y, i.geomtry.x], )
        return
    
ammenities = [malls, hawkercentres, schools]
ans = allscore([1,2,3,4,5], subzones, 5, ammenities)
ans


In [73]:
ans